In [1]:
import pandas as pd

In [2]:
census_data = pd.read_csv('census_data.csv')

In [3]:
census_data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [4]:
census_data['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
income_bracket = []
for i in range(len(census_data['income_bracket'])):
    if census_data['income_bracket'][i] == ' <=50K':
        income_bracket.append(0)
    else:
        income_bracket.append(1)

census_data['income_bracket'] = income_bracket

In [6]:
census_data['income_bracket']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        1
8        1
9        1
10       1
11       1
12       0
13       0
14       1
15       0
16       0
17       0
18       0
19       1
20       1
21       0
22       0
23       0
24       0
25       1
26       0
27       1
28       0
29       0
        ..
32531    0
32532    1
32533    1
32534    0
32535    0
32536    1
32537    0
32538    1
32539    1
32540    0
32541    0
32542    0
32543    0
32544    0
32545    1
32546    0
32547    0
32548    0
32549    0
32550    0
32551    0
32552    0
32553    0
32554    1
32555    0
32556    0
32557    1
32558    0
32559    0
32560    1
Name: income_bracket, Length: 32561, dtype: int64

In [7]:
census_data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


### Perform a Train Test Split on the Data

In [8]:
x_data = census_data.drop('income_bracket',axis=1)

In [9]:
y_data = census_data['income_bracket']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [11]:
census_data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [12]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [13]:
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size = 1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size = 1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size = 1000)
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size = 1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size = 1000)
race = tf.feature_column.categorical_column_with_vocabulary_list("race", ['White','Black'])
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ['Female','Male'])
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size = 1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [14]:
age  = tf.feature_column.numeric_column("age")
education_num  = tf.feature_column.numeric_column("education_num")
capital_gain  = tf.feature_column.numeric_column("capital_gain")
capital_loss  = tf.feature_column.numeric_column("capital_loss")
hours_per_week  = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [15]:
feat_cols = [age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [16]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train, batch_size = 10, num_epochs = None, shuffle = True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [ ]:
linear_model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

** Train your model on the data, for at least 5000 steps. **

In [ ]:
linear_model.train(input_fn = input_func,steps = 5000)

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [19]:
predict = tf.estimator.inputs.pandas_input_fn(x= X_test,batch_size = len(X_test),shuffle = False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [ ]:
predictions = list(linear_model.predict(input_fn = predict))

** Each item in your list will look like this: **

In [21]:
predictions[0]

{'logits': array([-1.0700988], dtype=float32),
 'logistic': array([0.2553843], dtype=float32),
 'probabilities': array([0.7446157, 0.2553843], dtype=float32),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [22]:
ids_prediction = [prediction['class_ids'][0] for prediction in predictions]
ids_prediction

[0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [23]:
from sklearn.metrics import classification_report

In [24]:
report = classification_report(y_test,ids_prediction)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      8161
           1       0.61      0.71      0.65      2585

   micro avg       0.82      0.82      0.82     10746
   macro avg       0.76      0.78      0.77     10746
weighted avg       0.83      0.82      0.83     10746

